In [1]:
import random
from faker import Faker
from datetime import datetime, timedelta
from random import choice, choices
import os
import csv
import pandas as pd

ModuleNotFoundError: No module named 'faker'

In [ ]:
fake = Faker()
# La clase ONG lleva la mayor parte de la lógica del programa
class ONG:
    # Lista que simula una red de ongs asociadas a la nuestra
    red_ongs = ["UnityWorks Foundation", "Solidaridad Internacional", "Global Harmony Alliance", "Manos del Futuro",
                "Migrant Hope Foundation", "Cambio Positivo", "KinderAid", "Acción Humanitaria", "Sonrisas del Mundo",
                "Hope for All", "New Beginnings", "Nuevo horizonte", "Geração Solidária", "Vida Nova", "Oikonomikí Drási",
                "Unidos pela Vida", "Ação Voluntária", "Horizontes Abiertos", "Corazón sin Fronteras", "Sostegno Vita",
                "Aiuto per il Futuro", "Esperanza para Todos", "To you foundation, Russia","Tzu Chi Foundation",
                "Yuwa Foundation", "The Bamboo Project","Ayuda Comunitaria", "Sociedad del Futuro",
                "Foundation for a Better World"]
    
    # Definimos el constructor de la clase
    def __init__(self):
        self.challenges_activos = []
        self.personas = []
        self.challenges_exitosos = []
        self.challenges_discontinuados = []
        self.red_ongs = ONG.red_ongs
        self.ongs_colaboradoras = []
        self.paises_colaboradores= []
        self.paises = Herramientas().paises
    # Nuestra ong puede crear challenges de recaudación de fondos o voluntariados. Los creamos al azar
    def crear_challenge(self,fecha_inicio):
        rand = random.random()
        if rand < 0.65:
             challenge = Donacion(fecha_inicio)
        elif rand < 0.85:
             challenge = VoluntariadoPresencial(fecha_inicio)
        elif rand < 0.94:
             challenge = VoluntariadoTraductor(fecha_inicio)    
        elif rand < 0.98:
             challenge = VoluntariadoAbogado(fecha_inicio)
        else:
             challenge = Challenge(fecha_inicio)
        self.challenges_activos.append(challenge)
        
        challenge.pais_destinatario = Herramientas().elegir_pais_al_azar(self.paises)
        return challenge
    # podemos enlistar las personas que se sumen a nuestros challenges
    def agregar_persona(self, persona):
         self.personas.append(persona)
    # Podemos quitar un challenge en particular si fuese necesario
    def remover_challenge(self, challenge):
        if challenge in self.challenges_activos:
            self.challenges_activos.remove(challenge)
        else:
            print("El challenge ya no está en la lista.")
    # Se pueden remover los datos de las personas del listado de la ONG, si a quien se intenta remover no está, tenemos un aviso
    def remover_persona(self, persona):
        if persona in self.personas:
            self.personas.remove(persona)
        else:
            print("La persona ya no está en la lista.")
    # Si un challenge alcanza su objetivo se remueve de la lista de challenges activos de la ONG y se pasa al listado
    # de challenges exitosos
    def mover_challenge_exitoso(self, challenge):
        if (challenge in self.challenges_activos) and (challenge.estado_objetivo == "Exitoso"): 
            self.challenges_activos.remove(challenge)
            self.challenges_exitosos.append(challenge)
        else:
            print("El challenge no está activo.")
                  
    # Algo similar sucede con los challenges que no alcanzan el objetivo
    def mover_challenge_discontinuado(self, challenge):      
        if (challenge in self.challenges_activos):
            self.challenges_activos.remove(challenge)
            self.challenges_discontinuados.append(challenge)
        else:
            print("El challenge no está activo.")
    # Si la ONG se enoja elimina todos los challenges activos al mismo tiempo.         
    def discontinuar_todos(self):
        for challenge in self.challenges_activos:
            self.mover_challenge_discontinuado(challenge)
        self.challenges_activos = []
    # Si un país colabora, siempre que haya al menos 2*factor challenges activos, automáticamente le da estado de "Exitoso"
    # a 2*factor challenges. factor es un número utilizado en el loop principal del programa, aumenta con el nivel de la ONG
    def recibir_ayuda_gubernamental(self, factor):
        if len(self.challenges_activos) > (2*factor):
            for i in range(2*factor):
                challenge_elegido = random.choice(self.challenges_activos)
                challenge_elegido.estado_objetivo = "Exitoso"
                self.paises_colaboradores.append(challenge_elegido.pais_destinatario)
                self.mover_challenge_exitoso(challenge_elegido)
    # Las ongs de nuestra red pueden colaborar. Su ayuda se determina en el loop principal, acá elegimos al azar cual.
    def recibir_ayuda_ongs(self):
        mi_lista = self.red_ongs
        ong_amiga = random.choice(mi_lista)
        self.ongs_colaboradoras.append(ong_amiga)
        
    # Método para la visualización de los atributos de clase. Desde ahora lo notaremos to_string
    def __str__(self):
        participantes = "\n".join([f"{p.nombre} {p.apellido}" for p in self.personas])
        return f"Challenges activos: {[challenge.id_challenge for challenge in self.challenges_activos]}\nParticipantes:\n{participantes}"
  
 # Superclase Challenge    
class Challenge:
    siguiente_id = 0

    def __init__(self,fecha_inicio):
        Challenge.siguiente_id += 1         # número incremental. Se actualiza con cada nuevo challenge, ayuda a la identificación. 
        self.fecha_inicio = fecha_inicio    # Fecha en que se inicia el challenge. Tiene que ser formato datetime
        self.anio_de_inicio = fecha_inicio.year     # Año en que inicia el challenge, segunda componente de la identificación.
        self.deadline = fecha_inicio + timedelta(days=21)  # Deadline del challenge, medido en días
        self.fecha_fin = None               # Fecha de finalización, se actualiza cuando el challenge deja de estar activo
        self.id_challenge =  str(self.anio_de_inicio) +"-"+ str(Challenge.siguiente_id) # id del challenge
        self.participantes = []                                # Lista de participantes de este challenge
        self.estado_objetivo = "En progreso"                 # Estado del challenge, se actualiza cuando deja de estar activo
        self.tipo_de_challenge = "Simulacro"    # La ONG hace simulacros y estudia la respuesta de sus potenciales voluntarios
        self.pais_destinatario = ''                    # País beneficiario de los challenges de la ONG
    
    # Es necesario agregar personas a los challenges. Solo instancias de la clase Persona pueden participar, o sus subclases.
    def agregar_participante(self, participante):
        if isinstance(participante, Persona):      
            self.participantes.append(participante)
            participante.challenge = self  # establecer el atributo challenge en la persona correspondiente
        else:
            raise ValueError("Solo se pueden agregar objetos de la clase Persona como participantes.")
        # Los simulacros terminan cuando 15 personas se hayan unido, se declara al challenge exitoso    
        if len(self.participantes) == 15:
            ong.mover_challenge_exitoso(self)         
    
          
    # Setters    # Son métodos para modificar los valores de los atributo de clase 
    def establecer_estado(self, estado_objetivo):
        self.estado_objetivo = estado_objetivo  
        
    def establecer_deadline(self, deadline):
        self.deadline = deadline
        
    def establecer_fecha_fin(self, fecha_fin):
        self.fecha_fin = fecha_fin
    # To_str    
    def __str__(self):
        # Creamos variables de tipo string para la fecha, son más amigable que las instancias datetime
        fecha_inicio_str = self.fecha_inicio.strftime("%d/%m/%Y")
        fecha_fin_str = self.fecha_fin.strftime("%d/%m/%Y") if self.fecha_fin else "No definido"
        # El método retorna una selección de atributos de la clase que queremos ver
        return f"Id challenge: {self.id_challenge}\nTipo de challenge: {self.tipo_de_challenge}\n\
        Pais Destino: {self.pais_destinatario}\n\
        Participantes {[participante.nombre_apellido() for participante in self.participantes]}\n\
        Estado objetivo: {self.estado_objetivo}\nFecha de inicio: {fecha_inicio_str}\n\
        Fecha de finalización: {fecha_fin_str}"

# Clase hija de Challenge, Donacion
class Donacion(Challenge):
   
    def __init__(self,fecha_inicio):
        super().__init__(fecha_inicio)   # Lamamos al constructor de la clase padre con estos argumentos
        self.tipo_de_challenge = "Donacion"             
        self.monto = 0                      # monto acumulado, se actualiza con cada donación
        self.monto_objetivo = 100000        # monto objetivo, cambia el valor para cada nuevo challenge
        
        # Establecemos el monto objetivo con diferentes probabilidades
        montos = [5000, 10000, 15000, 20000, 25000, 30000]
        probabilidades = [0.2, 0.2, 0.3, 0.15, 0.10, 0.05]
        self.monto_objetivo = random.choices(montos, probabilidades)[0]

        # Diccionario de plazos en días para cada monto objetivo
        plazos = {5000: 10, 10000: 15, 15000: 20, 20000: 25, 25000: 30, 30000: 40}

        # Establecemos el deadline en base al monto objetivo elegido
        self.deadline = fecha_inicio + timedelta(days=plazos[self.monto_objetivo])
        
    # cada participante que se agrega al challenge hace una donación al azar entre 10 y 1000 dólares 
    def agregar_participante(self, participante):
        super().agregar_participante(participante)
        self.monto += random.randint(10, 1000)
        if self.monto >= self.monto_objetivo:    
            self.establecer_estado("Exitoso")
     
    # to_str  
    def __str__(self):
        fecha_deadline_str = self.deadline.strftime("%d/%m/%Y")
        fecha_inicio_str = self.fecha_inicio.strftime("%d/%m/%Y")
        fecha_fin_str = self.fecha_fin.strftime("%d/%m/%Y") if self.fecha_fin else "No definido"
        
        return f"Id challenge: {self.id_challenge}\nTipo de challenge: {self.tipo_de_challenge}\n\
        Pais Destino: {self.pais_destinatario}\n\
        Participantes {[participante.nombre_apellido() for participante in self.participantes]}\n\
        Monto objetivo: {self.monto_objetivo}\nDeadline : {fecha_deadline_str}\nMonto: {self.monto}\n\
        Estado objetivo: {self.estado_objetivo}\nFecha de inicio: {fecha_inicio_str}\n\
        Fecha de finalización: {fecha_fin_str}"
            
# Segunda clase hija de Challenge    
class VoluntariadoTraductor(Challenge):
    
    def __init__(self,fecha_inicio,):
        super().__init__(fecha_inicio)   # Lamamos al constructor de la clase padre con estos argumentos
        self.tipo_de_challenge = "Traductor"  
    # Este challenge tiene un máximo de 10 participantes
    def agregar_participante(self, participante):
        if len(self.participantes) < 10:                   # Si hay menos de 10 participantes unidos
            super().agregar_participante(participante)    # utiliza el método agregar_participante de la superclase
        else:                                             # Sino:
            if self.estado_objetivo != "Discontinuado":     # chequea si el estado_objetivo no es discontinuado
                self.establecer_estado("Exitoso")           # si no lo es, se declara a este challenge exitoso.
    
    # to_str  
    def __str__(self):
        fecha_inicio_str = self.fecha_inicio.strftime("%d/%m/%Y")
        fecha_fin_str = self.fecha_fin.strftime("%d/%m/%Y") if self.fecha_fin else "No definido"
        
        return f"Id challenge: {self.id_challenge}\nTipo de challenge: {self.tipo_de_challenge}\n\
        Pais Destino: {self.pais_destinatario}\n\
        Participantes {[participante.nombre_apellido() for participante in self.participantes]}\n\
        Estado objetivo: {self.estado_objetivo}\nFecha de inicio: {fecha_inicio_str}\n\
        Fecha de finalización: {fecha_fin_str}"        

# Tercera clase hija de Challenge    
class VoluntariadoAbogado(Challenge):
    
    def __init__(self,fecha_inicio,):
        super().__init__(fecha_inicio)   # Lamamos al constructor de la clase padre con estos argumentos
        self.tipo_de_challenge = "Abogado"      
          
    # Idéntico al de la clase Voluntariado traductor, excepto en la cantidad de participantes.
    def agregar_participante(self, participante):
        if len(self.participantes) < 8:
            super().agregar_participante(participante)
        else:
            if self.estado_objetivo != "Discontinuado":
                self.establecer_estado("Exitoso")
            
    # Setters
    def establecer_deadline(self, deadline):
        self.deadline = deadline
        
    def establecer_fecha_fin(self, fecha_fin):
        self.fecha_fin = fecha_fin         
           
    
    # to_string
    def __str__(self):
        fecha_inicio_str = self.fecha_inicio.strftime("%d/%m/%Y")
        fecha_fin_str = self.fecha_fin.strftime("%d/%m/%Y") if self.fecha_fin else "No definido"
        
        return f"Id challenge: {self.id_challenge}\nTipo de challenge: {self.tipo_de_challenge}\n\
        Pais Destino: {self.pais_destinatario}\n\
        Participantes {[participante.nombre_apellido() for participante in self.participantes]}\n\
        Estado objetivo: {self.estado_objetivo}\nFecha de inicio: {fecha_inicio_str}\n\
        Fecha de finalización: {fecha_fin_str}"      
    
# Cuarta y última subclase de Challenge
class VoluntariadoPresencial(Challenge):
    
    def __init__(self, fecha_inicio):
        super().__init__(fecha_inicio)   # Lamamos al constructor de la clase padre con estos argumentos
        self.tipo_de_challenge = "Presencial"
            
    
    def agregar_participante(self, participante):
        if len(self.participantes) < 35:
            super().agregar_participante(participante)
        else:
            if self.estado_objetivo != "Discontinuado":
                self.establecer_estado("Exitoso")          
    
    def __str__(self):
        fecha_inicio_str = self.fecha_inicio.strftime("%d/%m/%Y")
        fecha_fin_str = self.fecha_fin.strftime("%d/%m/%Y") if self.fecha_fin else "No definido"
        
        return f"Id challenge: {self.id_challenge}\nTipo de challenge: {self.tipo_de_challenge}\n\
        Pais Destino: {self.pais_destinatario}\n\
        Participantes {[participante.nombre_apellido() for participante in self.participantes]}\n\
        Estado objetivo: {self.estado_objetivo}\nFecha de inicio: {fecha_inicio_str}\n\
        Fecha de finalización: {fecha_fin_str}"        
    
# Superclase Persona
class Persona:
    # Queda para el futuro cambiar los atributos a tipo string y modificarlos con Faker() a través de métodos. 
    def __init__(self):
        fake = Faker()          # El objeto Faker será el que nos de los valores de los atributos 
        self.nombre = fake.first_name()      
        self.apellido = fake.last_name()
        self.edad = self.generar_edad()
        self.mail = fake.email()
        id_num = fake.random_number(digits=8)          # Numero de identificación, tiene 8 dígitos seleccionados al azar
        id_letra = chr(fake.random_int(min=65, max=90))   # Elegimos un char al azar entre A y Z
        self.id_nacional = id_letra + str(id_num).zfill(8) #Componemos la identificación con los dos valores anteriores
        self.nacionalidad = fake.country()      # Creamos un nombre de país
        self.challenge = None                   # El valor por defecto del challenge asociado a esta persona es None
        self.tipo_challenge = "Ninguno"         # por defecto el challenge no tiene ningún tipo asociado
   
    # No queremos que las edades estén igualmente distribuídas, así que creamos edades condicionando las probabilidades   
    def generar_edad(self):
        rand = random.random()
        if rand < 0.55:
            return random.randint(18, 40)
        elif rand < 0.85:
            return random.randint(41, 60)
        elif rand < 0.95:
            return random.randint(61, 70)
        else:
            return random.randint(71, 92)
        
    def __str__(self):
        return f"Nombre: {self.nombre} {self.apellido}\nEdad: {self.edad}\nE-mail: {self.mail}\nID Nacional:\
        {self.id_nacional}\nNacionalidad: {self.nacionalidad}\nTipo de Challenge: {self.tipo_challenge}"
       
# Subclase CreadorDeChallenge, sus instancias pueden crear challenges    
class CreadorDeChallenge(Persona):
    def __init__(self, ong, fecha_inicio):       
        super().__init__()
        fecha_de_creacion = fecha_inicio          # Fecha en que se crea el challenge    
        anio_de_inicio = fecha_de_creacion.year   # datetime
        self.challenge_creado = ong.crear_challenge(fecha_de_creacion)  # pide a ong crear un challenge 
        self.challenge_creado.agregar_participante(self)  # una vez que crea el challenge el mismo se une
        ong.agregar_persona(self)   # pide ser agregado en lalista personas 
    # Crea challenges al azar            
    def crear_challenge(self):
        if random.random() < 0.5:
            challenge = VoluntariadoPresencial()
        else:
            challenge = Challenge()
        self.challenges_activos.append(challenge)
        return challenge    
    # juntamos el nombre y apellido     
    def nombre_apellido(self):
        return f"{self.nombre} {self.apellido}"    

    def __str__(self):
        return f"Nombre: {self.nombre} {self.apellido}\nEdad: {self.edad}\nE-mail: {self.mail}\nID Nacional:\
        {self.id_nacional}\nNacionalidad: {self.nacionalidad}\nTipo de Challenge: {self.challenge_creado.tipo_de_challenge}"    
# Subclase Seguidor, hereda de Persona 
class Seguidor(Persona):
    def __init__(self):
        super().__init__()
        self.challenges = []
        self.tipo_challenge = None
    # Las instancias de esta clase pueden unirse a los challenges ya creados    
    def unirse_challenge(self, challenge):
        if challenge in self.challenges:
            print("Ya te uniste a este challenge.")
        else:
            challenge.agregar_participante(self)
            self.challenges.append(challenge)
            self.tipo_challenge = challenge.tipo_de_challenge      # Toma el tipo de challenge al que se une

    def nombre_apellido(self):
        return f"{self.nombre} {self.apellido}"

    def buscar_challenges(self, lista_challenges):
        for challenge in lista_challenges:
            print(f"Challenge {challenge.numero}: {len(challenge.participantes)} participantes.")
            
            
    def unirse_challenge_aleatorio(self, ong):
        challenges = ong.challenges_activos
        n_challenges = len(challenges)
        if n_challenges == 0:
            print("No hay challenges disponibles.")
            return
        idx = random.randint(0, n_challenges-1)
        challenge = challenges[idx]
        self.unirse_challenge(challenge)
       
    def __str__(self):
        return f"Nombre: {self.nombre} {self.apellido}\nEdad: {self.edad}\nE-mail: {self.mail}\nID Nacional:\
        {self.id_nacional}\nNacionalidad: {self.nacionalidad}\nTipo de Challenge: {self.tipo_challenge}"     

In [ ]:
# La clase Herramientas es una clase auxiliar que pone a disposición una serie recursos utilizados en nuestra simulación

class Herramientas:
    # La lista países es utilizada para simular los países que se puedan unir a las causas de nuestra ong
    paises = ['UNITED STATES', 'RUSSIA', 'GERMANY', 'SYRIAN ARAB REPUBLIC', 'COLOMBIA', 'UNITED KINGDOM', 'SPAIN',
 'CANADA', 'AFGHANISTAN', 'AUSTRALIA', 'CHILE', 'JAPAN', 'SWEDEN', 'PERU', 'IRAQ', 'NETHERLANDS', 'MALAYSIA', 'BRAZIL',
 'SWITZERLAND', 'BELGIUM', 'UGANDA', 'PORTUGAL', 'ETHIOPIA', 'NEW ZEALAND', 'ECUADOR', 'THAILAND', 'SOUTH AFRICA',
 'ITALY', 'DENMARK', 'SINGAPORE', 'AUSTRIA', 'FRANCE', 'JORDAN', 'FINLAND', 'ISRAEL', 'IRELAND', 'BELARUS', 'MOZAMBIQUE',
 'MALTA', 'NORWAY','PAPUA NEW GUINEA', 'HUNGARY', 'CHAD', 'ZAMBIA', 'MACAO', 'SLOVENIA', 'ANGOLA', 'LUXEMBOURG', 'PANAMA']
    
    def __init__(self):
        self.paises = Herramientas.paises     # Pasamos la lista como atributo de clase, para poder utilizarla en esta.
    # Elegimos países al azar, asignamos 3% de probabilidades de elección de para #EEUU y Alemania, 2% para el resto. 
    def elegir_pais_al_azar(self, paises):
        probabilidades = [0.03 if pais in ['UNITED STATES', 'GERMANY'] else 0.02 for pais in paises]
        
        return random.choices(paises, weights=probabilidades)[0]    # retornamos un país al azar, según los pesos asignados.
    # Método que guarda todas las instancias de Persona y clases hijas en un archivo csv
    def guardar_instancias(self, lista):
        with open('instancias2.csv', mode='w', newline='') as archivo:
            writer = csv.writer(archivo)
            writer.writerow(['nombre', 'apellido', 'edad', 'correo electrónico', 'id_nacional', 'nacionalidad',
                             'tipo_de_challenge', 'categoría_persona'])
            for inst in instancias:
                if isinstance(inst, CreadorDeChallenge):
                    # Si es una instancia de CreadorDeChallenge, escribir los datos correspondientes
                    writer.writerow([inst.nombre, inst.apellido, inst.edad, inst.mail, inst.id_nacional,
                                 inst.nacionalidad, inst.challenge_creado.tipo_de_challenge, str(type(inst).__name__)])
                elif isinstance(inst, Seguidor):
                    # Si es una instancia de Seguidor, escribir los datos correspondientes
                    writer.writerow([inst.nombre, inst.apellido, inst.edad, inst.mail, inst.id_nacional,
                                 inst.nacionalidad, inst.tipo_challenge , str(type(inst).__name__)])
                elif isinstance(inst, Persona):
                    # Si es una instancia de Persona, escribir los datos correspondientes
                     writer.writerow([inst.nombre, inst.apellido, inst.edad, inst.mail, inst.id_nacional,
                                  inst.nacionalidad, '', str(type(inst).__name__)])
                else:
                    # Si es otra instancia, ignorarla
                    pass
    
    # El método estático no necesita acceso a los atributos de un constructor.
    # En Python los métodos estáticos se señalan con un decorado no va incluído en la firma como en Java
    @staticmethod
    def actualizar_fecha_simulada(inicio_sim, fecha_sim, inicio_crono, factor_acel ):
        tiempo_transcurrido = datetime.now() - inicio_crono
        tiempo_transcurrido_segundos = tiempo_transcurrido.total_seconds()
        tiempo_simulado = timedelta(seconds=tiempo_transcurrido_segundos * factor_acel)
        fecha_simulada = inicio_sim + tiempo_simulado
        
        return fecha_simulada

In [ ]:
%%time                                               
# Simulamos con datetime las fechas que necesitaremos
inicio_simulacion = datetime(2021, 1, 1, 10, 30, 0)    
fecha_simulada = datetime(2021, 1, 1, 10, 30, 0)
fin_simulacion = datetime(2022, 1,30, 10, 30, 0)

ong = ONG()                   # Instanciamos nuestro objeto ONG
h = Herramientas()            # Guardamos nuestros recursos adicionales en la variable h para una mayor comodidad
instancias = []               # Creamos la lista que va a guardar cada instancia creada en el loop principal.
total_challenges = 0          # Llevaremos un conteo de challenges que determinará el nivel de la ONG
nivel = 1                     # el nivel simula la fama de la ONG, a mayor nivel mayor cantidad de recursos serán creados

# Loop principal del programa, se itera hasta que se haya alcanzado la fecha fin de simulación, comenzamos en la fecha
# incio_simulación. Avanzaremos un día en cada iteración.
while fecha_simulada <= fin_simulacion:
    if (fecha_simulada.date().day == 1) or (len(ong.challenges_activos) == 0):
    # Instanciamos objetos de la clase Challenge, van a ser nuestros challenges iniciales de cada mes
        for i in range(0,4 + (nivel*3)):    # Se irá incrementando a medida que suba el nivel de la ONG
            ong.crear_challenge(fecha_simulada)
            total_challenges += 1
     
    # Recibir ayuda de ongs
    rand = random.random()
    if rand < (0.1*(0.5 *nivel)):       # probabilidad de ejecutar el if, comienza en 5 % y aumenta 5% por cada nivel
        ong.recibir_ayuda_ongs()             # Si se ejecuta la sentencia if, recibimos ayuda de una ong aliada
        for i in range(10 +(2*nivel) ):         # El rango del for aumenta con cada nivel, habrá más instancias 
            s = Seguidor()                      # de seguidor por cada nivel. Se crean las instancias
            s.unirse_challenge_aleatorio(ong)   # Se unen a los challenges activos
            instancias.append(s)                # Guardamos el objeto de la clase Seguidor en la lista de instancias   
              
    # Creamos i instancias de la clase Persona y sus sublclases, al azar. Las añadimos a la lista instancias
    for i in range(4 + (2*nivel)):             # Por cada nivel se crearán dos personas más por día.
        rand = random.random()
        if rand < 0.46:
            p = Persona()
            instancias.append(p)
        elif rand < 0.56:
            c = CreadorDeChallenge(ong,fecha_simulada)
            instancias.append(c)
            total_challenges += 1   
        else:
            s = Seguidor()
            s.unirse_challenge_aleatorio(ong)
            instancias.append(s)    

    # Creamos j instancias de la clase Seguidor que ayudarán a cumplir los challenges        
    for j in range(5*(nivel)):
        s = Seguidor()
        s.unirse_challenge_aleatorio(ong)
        instancias.append(s)                  # y luego las añadimos a la lista instancias.
    
    for challenge in ong.challenges_activos:                 # Por cada challenge activo...
        if challenge.estado_objetivo == 'Exitoso':           # si su estado de cumplimiento del objetivo es "Exitoso"
            challenge.establecer_fecha_fin(fecha_simulada)   # se establece su atributo fecha_fin con la fecha simulada
            ong.mover_challenge_exitoso(challenge)           # se mueve el challenge a la lista de challenges exitosos.
    for challenge in ong.challenges_activos:                 # Algo similar hacemos para discontinuar los challenges
        if (challenge.deadline < fecha_simulada):            # que no cumplieron con su deadline.
            challenge.fecha_fin = fecha_simulada
            ong.mover_challenge_discontinuado(challenge)
            
    # Recibir ayuda gubernamental
    if rand < 0.1:                                           # Existe un 10 % de probabilidad de ocurrencia
        ong.recibir_ayuda_gubernamental(nivel)               # si se da ese 10 % la ong recibe ayuda de algún país.
            
    # Condiciones para la suba de nivel de la ong                          
    if total_challenges < 0:
               total_challenges = 0
    if total_challenges < 200:
        nivel = 1
    elif total_challenges >= 200 and total_challenges < 500:
        nivel = 2
    else:
        nivel = 3      
    # Aumentamos en un día la fecha simulada, para poder alcanzar en algún momento la fecha tope del loop. 
    fecha_simulada = fecha_simulada + timedelta(days=1)
# Guardamos todas las instancias creadas
h.guardar_instancias(instancias)         

In [ ]:
# Creamos una lista vacía para almacenar las tuplas
challenges_exitos = []
# Iteramos sobre la lista de objetos
for challenge in ong.challenges_exitosos:

    # Obtenemos el valor de los atributos y agregarlos a una tupla
    challenge_info = (challenge.id_challenge, challenge.tipo_de_challenge, 'exitoso',
                      challenge.fecha_inicio, challenge.fecha_fin, challenge.pais_destinatario)
    
    # Agregamos la tupla a la lista de challenges que tuvieron exito
    challenges_exitos.append(challenge_info)
      
# Calculamos el ratio de éxitos 
ratio_de_exito = len(ong.challenges_exitosos)/(len(ong.challenges_exitosos) + len(ong.challenges_discontinuados))
ratio_de_exito = round(ratio_de_exito,2)
print(f"Ratio de exito {ratio_de_exito}")
print("*"*100)
print(challenges_exitos)
print("*"*100)

# Creamos una lista vacía para almacenar las tuplas
challenges_disc = []

# Iteraramos sobre la lista de objetos
for challenge in ong.challenges_discontinuados:

    # Obtenemos el valor de los atributos y agregarlos a una tupla
    challenge_info_disc = (challenge.id_challenge, challenge.tipo_de_challenge, 'discontinuado',
                           challenge.fecha_inicio, challenge.fecha_fin, challenge.pais_destinatario)    
    # Agregamos la tupla a la lista de información de challenges
    challenges_disc.append(challenge_info_disc)
print(f"Challenges disc {challenges_disc}")   

# Creamos lista vacía para almacenar las tuplas
challenges_act = []
for challenge in ong.challenges_activos:
    # Obtenemos el valor de los atributos y agregarlos a una tupla
    challenges_info_act = (challenge.id_challenge, challenge.tipo_de_challenge, 'en curso',
                           challenge.fecha_inicio, challenge.fecha_fin, challenge.pais_destinatario)
    # Agregamos la tupla a la lista de información de challenges
    challenges_act.append(challenges_info_act)
    print("*"*100)
    print(f"Challenges disc {challenges_act}")
    

## Creamos data frames para recuperar todos los datos creados 

#### Data frames de challenges

In [ ]:
# Creamos un DataFrame a partir de la lista de tuplas para los challenges exitosos
df_ex = pd.DataFrame(challenges_exitos, columns=['id_challenge', 'tipo_de_challenge', 'resultado', 
                                                 'fecha_inicio', 'fecha_fin','pais_destinatario'])

In [ ]:
# Lo mismo para los challenges discontinuados
df_disc = pd.DataFrame(challenges_disc, columns=['id_challenge', 'tipo_de_challenge', 'resultado', 
                                                 'fecha_inicio', 'fecha_fin', 'pais_destinatario'])

In [ ]:
# Lo mismo para los challenges activos
df_act = pd.DataFrame(challenges_act, columns=['id_challenge', 'tipo_de_challenge', 'resultado', 
                                                 'fecha_inicio', 'fecha_fin','pais_destinatario'])

In [ ]:
# Unimos los data frames en uno solo
df_res = pd.concat([df_disc,df_ex,df_act]).reset_index(drop=True)

In [ ]:
directorio = "simulacion"

# Creamos la carpeta si no existe
if not os.path.exists(directorio):
    os.makedirs(directorio)

In [ ]:
# Definimos la ruta y el nombre del archivo CSV
nombre_archivo = "anio2021_resultados.csv"
ruta_archivo = os.path.join(carpeta, nombre_archivo)    
# Guardamos los datos
df_res.to_csv(ruta_archivo,index = False)

In [ ]:
ongs_colab = pd.DataFrame(ong.ongs_colaboradoras, columns=['ongs_colab'])

In [ ]:
# Definimos la ruta y el nombre del archivo CSV
nombre_archivo = "ongs_colab.csv"
ruta_archivo = os.path.join(carpeta, nombre_archivo)    
# Guardamos los datos
ongs_colab.to_csv(ruta_archivo,index = False)

In [ ]:
paises_colab = pd.DataFrame(ong.paises_colaboradores, columns=['paises_colab'])

In [ ]:
# Definimos la ruta y el nombre del archivo CSV
nombre_archivo = "paises_colab.csv"
ruta_archivo = os.path.join(carpeta, nombre_archivo)    
# Guardamos los datos
paises_colab.to_csv(ruta_archivo,index = False)

## Comprobamos que hayan sido guardados correctamente

In [ ]:
#df = pd.read_csv('simulacion\\instancias2.csv', encoding = 'latin')

In [ ]:
#df = pd.read_csv('simulacion\\paises_colab.csv')

In [ ]:
#df = pd.read_csv('simulacion\\ongs_colab.csv')